In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
from models import *
from utils import *
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Concatenate, Add, Conv2DTranspose, Input, Lambda, Reshape, LeakyReLU, Flatten, ReLU, Multiply, Subtract, Conv3D
from tensorflow.keras.models import *
import glob
import random
#import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops
import time
import pandas as pd

In [ ]:
#train U-Net for each subsampled dataset
for angle in [64, 128, 256, 512, 1024, 2048]:

    save_path = os.path.abspath("./model_weights/") + f"{angle}"
    
    #Instantiate optimzers:
    lr=1e-4
    beta_1 = 0.9
    beta_2 = 0.999

    generator_optimizer = keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2)

    # load model

    model = U_Net()
    #gen.summary()
    model.compile(optimizer=generator_optimizer, loss=loss_UNet)
    data_gen = load_batch(train_list, base_dir, batchsize=batchsize, angle=angle, shape=(256, 256))

    callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=save_path + '/model_{epoch:02d}.h5', verbose=1, period=10, save_weights_only=True), 
                tf.keras.callbacks.CSVLogger(save_path + '/log.csv', append=True, separator=','), 
                tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1), 
                tf.keras.callbacks.TensorBoard(save_path + 'log/{}'.format(time.time()))]

    model.fit(data_gen, epochs=75, steps_per_epoch=len(train_list)//batchsize, callbacks=callbacks)